In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import re

# Configuration du navigateur
options = Options()
options.add_argument("--start-maximized")

# Initialiser le navigateur
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)  # Attente explicite de 10 secondes maximum

# Fonction pour mettre en évidence un élément avec une bordure rouge
def highlight_element(element, duration=0.5):
    # Faire défiler jusqu'à l'élément pour qu'il soit visible
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
    time.sleep(0.2)  # Attendre que le défilement soit terminé
    
    # Ajouter une bordure rouge et un fond rouge transparent
    original_style = element.get_attribute('style')
    driver.execute_script("""
    arguments[0].setAttribute('style', arguments[1] + 
        '; border: 3px solid red; background: rgba(255, 0, 0, 0.2)');
    """, element, original_style)
    
    time.sleep(duration)  # Maintenir la mise en évidence
    
    # Restaurer le style original
    driver.execute_script("""
    arguments[0].setAttribute('style', arguments[1]);
    """, element, original_style)

# Fonction pour afficher un message de progression à l'écran
def show_progress_message(message):
    # Créer ou mettre à jour l'élément de message de progression
    driver.execute_script("""
        var progressMsg = document.getElementById('scraping-progress-msg');
        if (!progressMsg) {
            progressMsg = document.createElement('div');
            progressMsg.id = 'scraping-progress-msg';
            progressMsg.style.position = 'fixed';
            progressMsg.style.top = '10px';
            progressMsg.style.left = '10px';
            progressMsg.style.backgroundColor = 'rgba(0, 0, 0, 0.8)';
            progressMsg.style.color = 'white';
            progressMsg.style.padding = '10px';
            progressMsg.style.borderRadius = '5px';
            progressMsg.style.zIndex = '9999';
            progressMsg.style.fontSize = '16px';
            progressMsg.style.fontWeight = 'bold';
            document.body.appendChild(progressMsg);
        }
        progressMsg.textContent = arguments[0];
    """, message)

# Liste pour stocker toutes les données
all_car_data = []

try:
    # Accéder au site web
    driver.get("https://www.autoscout24.fr/")
    print("Page chargée avec succès")
    show_progress_message("Page chargée avec succès")
    time.sleep(3)
    
    # Gérer le popup de consentement spécifique à AutoScout24
    try:
        # Attendre que le popup de consentement apparaisse
        consent_popup = wait.until(EC.presence_of_element_located((By.ID, "as24-cmp-popup")))
        print("Popup de consentement détecté")
        show_progress_message("Popup de consentement détecté")
        
        # Accepter tous les cookies (bouton "Accepter tout")
        accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'consent-accept')]")))
        highlight_element(accept_button)
        accept_button.click()
        print("Cookies acceptés")
        show_progress_message("Cookies acceptés")
        time.sleep(2)
    except Exception as e:
        print(f"Erreur lors de la gestion du consentement: {e}")
        show_progress_message(f"Erreur: {e}")
    
    # Attendre que le bouton de recherche soit cliquable
    search_button = wait.until(EC.element_to_be_clickable((By.ID, "search-mask-search-cta")))
    print("Bouton de recherche trouvé")
    show_progress_message("Bouton de recherche trouvé")
    
    # Faire défiler jusqu'au bouton si nécessaire
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", search_button)
    time.sleep(1)
    
    # Mettre en évidence et cliquer sur le bouton
    highlight_element(search_button)
    search_button.click()
    print("Clic sur le bouton de recherche effectué")
    show_progress_message("Recherche de voitures en cours...")
    
    # Attendre que la page de résultats se charge
    wait.until(EC.url_contains("lst"))
    print("Page de résultats chargée")
    show_progress_message("Page de résultats chargée")
    
    # Boucle pour parcourir les 20 premières pages
    page_count = 1
    max_pages = 20
    current_url = driver.current_url
    
    while page_count <= max_pages:
        print(f"\n===== TRAITEMENT DE LA PAGE {page_count} =====")
        show_progress_message(f"TRAITEMENT DE LA PAGE {page_count}/{max_pages}")
        
        # Attendre que le conteneur principal soit chargé
        print("Recherche du conteneur principal ListPage_main___0g2X...")
        main_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "main.ListPage_main___0g2X")))
        highlight_element(main_container, 1)
        print(f"Conteneur principal de la page {page_count} trouvé")
        
        # Récupérer tous les articles (voitures) dans le conteneur principal
        articles = main_container.find_elements(By.CSS_SELECTOR, "article")
        print(f"Nombre d'articles (voitures) trouvés sur la page {page_count}: {len(articles)}")
        show_progress_message(f"Page {page_count}/{max_pages}: {len(articles)} voitures trouvées")
        
        # Parcourir tous les articles et extraire leurs attributs
        for i, article in enumerate(articles):
            progress_msg = f"Page {page_count}/{max_pages}: Extraction voiture {i+1}/{len(articles)}"
            print(f"Extraction des données de l'article {i+1} de la page {page_count}...")
            show_progress_message(progress_msg)
            
            # Faire défiler vers l'article
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", article)
            time.sleep(0.3)  # Petit délai pour le défilement fluide
            
            # Mettre en évidence l'article traité
            highlight_element(article, 0.3)
            
            # Récupérer tous les attributs de l'élément article
            attributes = driver.execute_script(
                "var items = {}; for (var i = 0; i < arguments[0].attributes.length; i++) { items[arguments[0].attributes[i].name] = arguments[0].attributes[i].value }; return items;", 
                article
            )
            
            # Extraire le contenu visible
            try:
                # Titre de la voiture
                title_element = article.find_element(By.CSS_SELECTOR, "h2")
                highlight_element(title_element, 0.2)
                title = title_element.text
            except:
                try:
                    title_element = article.find_element(By.CSS_SELECTOR, "h3")
                    highlight_element(title_element, 0.2)
                    title = title_element.text
                except:
                    title = "Titre non trouvé"
            
            try:
                # Prix
                price_element = article.find_element(By.CSS_SELECTOR, "span.price")
                highlight_element(price_element, 0.2)
                price = price_element.text
            except:
                try:
                    price_element = article.find_element(By.CSS_SELECTOR, "p[data-item-name='price']")
                    highlight_element(price_element, 0.2)
                    price = price_element.text
                except:
                    price = "Prix non trouvé"
            
            try:
                # Caractéristiques
                specs_elements = article.find_elements(By.CSS_SELECTOR, "ul[data-item-name='vehicle-highlights'] li")
                specs = []
                for spec_element in specs_elements:
                    highlight_element(spec_element, 0.1)
                    specs.append(spec_element.text)
            except:
                specs = []
            
            # Créer un dictionnaire avec toutes les données
            car_data = {
                "page": page_count,
                "position": i + 1,
                "id": attributes.get("id", ""),
                "class": attributes.get("class", ""),
                "data_attributes": {k: v for k, v in attributes.items() if k.startswith("data-")},
                "visible_content": {
                    "title": title,
                    "price": price,
                    "specs": specs
                }
            }
            
            # Extraire les identifiants et les valeurs importantes des données
            if "data-guid" in attributes:
                car_data["guid"] = attributes["data-guid"]
            if "data-vehicle-type" in attributes:
                car_data["vehicle_type"] = attributes["data-vehicle-type"]
            if "data-price" in attributes:
                car_data["price_value"] = attributes["data-price"]
            if "data-make" in attributes:
                car_data["make"] = attributes["data-make"]
            if "data-model" in attributes:
                car_data["model"] = attributes["data-model"]
            if "data-mileage" in attributes:
                car_data["mileage"] = attributes["data-mileage"]
            if "data-fuel-type" in attributes:
                car_data["fuel_type"] = attributes["data-fuel-type"]
            if "data-first-registration" in attributes:
                car_data["first_registration"] = attributes["data-first-registration"]
            
            # Ajouter l'URL si disponible via un lien
            try:
                url_element = article.find_element(By.CSS_SELECTOR, "a")
                car_data["url"] = url_element.get_attribute("href")
            except:
                car_data["url"] = ""
            
            # Ajouter l'image si disponible
            try:
                img_element = article.find_element(By.CSS_SELECTOR, "img")
                car_data["image_url"] = img_element.get_attribute("src")
            except:
                car_data["image_url"] = ""
            
            all_car_data.append(car_data)
            print(f"Données de l'article {i+1} de la page {page_count} extraites")
        
        # Enregistrer les données après chaque page
        with open(f"autoscout24_data_page_{page_count}.json", "w", encoding="utf-8") as json_file:
            json.dump(all_car_data, json_file, ensure_ascii=False, indent=2)
        
        print(f"Données de la page {page_count} enregistrées")
        show_progress_message(f"Page {page_count}/{max_pages} complète - {len(all_car_data)} voitures au total")
        
        # Vérifier s'il y a une page suivante
        if page_count < max_pages:
            try:
                # Trouver le bouton "Suivant"
                next_button = None
                
                # Plusieurs tentatives pour trouver le bouton suivant
                try:
                    next_button = wait.until(EC.element_to_be_clickable((
                        By.XPATH, "//button[@aria-label='Aller à la page suivante']")))
                except:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((
                            By.XPATH, "//button[contains(., 'Suivant')]")))
                    except:
                        try:
                            next_button = wait.until(EC.element_to_be_clickable((
                                By.CSS_SELECTOR, "li.prev-next button")))
                        except:
                            next_button = wait.until(EC.element_to_be_clickable((
                                By.XPATH, "//svg[contains(@class, 'chevron-right')]/parent::button")))
                
                if next_button:
                    # Faire défiler jusqu'au bouton suivant
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_button)
                    time.sleep(1)
                    
                    # Mettre en évidence et cliquer sur le bouton suivant
                    highlight_element(next_button, 1)
                    print(f"Bouton 'Suivant' trouvé. Passage à la page {page_count + 1}...")
                    show_progress_message(f"Navigation vers la page {page_count + 1}...")
                    next_button.click()
                    
                    # Attendre que la nouvelle page se charge
                    time.sleep(3)
                    
                    # Vérifier que la page a bien changé
                    new_url = driver.current_url
                    if new_url != current_url:
                        current_url = new_url
                        print(f"URL actuelle après clic: {current_url}")
                        page_count += 1
                    else:
                        print("L'URL n'a pas changé après le clic, tentative supplémentaire...")
                        next_button.click()
                        time.sleep(3)
                        new_url = driver.current_url
                        if new_url != current_url:
                            current_url = new_url
                            page_count += 1
                        else:
                            print("Impossible de passer à la page suivante.")
                            break
                else:
                    print("Bouton 'Suivant' non trouvé.")
                    break
            except Exception as e:
                print(f"Impossible de passer à la page suivante: {e}")
                show_progress_message(f"Erreur de navigation: {e}")
                break
        else:
            print(f"Nombre maximum de pages ({max_pages}) atteint.")
            show_progress_message(f"Fin de l'extraction - {max_pages} pages traitées")
            break
    
    # Enregistrer toutes les données dans un fichier JSON final
    with open("autoscout24_complete_data.json", "w", encoding="utf-8") as json_file:
        json.dump(all_car_data, json_file, ensure_ascii=False, indent=2)
    
    print(f"\nToutes les données ont été extraites et enregistrées dans 'autoscout24_complete_data.json'")
    print(f"Nombre total de voitures extraites: {len(all_car_data)}")
    show_progress_message(f"TERMINÉ! {len(all_car_data)} voitures extraites au total")
    
    # Afficher un résumé des données
    makes_count = {}
    models_count = {}
    fuel_types = {}
    registration_years = {}
    
    for car in all_car_data:
        make = car.get("make", "Inconnu")
        model = car.get("model", "Inconnu")
        fuel = car.get("fuel_type", "Inconnu")
        reg_year = car.get("first_registration", "Inconnu")
        
        makes_count[make] = makes_count.get(make, 0) + 1
        models_count[model] = models_count.get(model, 0) + 1
        fuel_types[fuel] = fuel_types.get(fuel, 0) + 1
        registration_years[reg_year] = registration_years.get(reg_year, 0) + 1
    
    # Afficher un résumé à l'écran
    summary_html = f"""
    <div style="background-color: #333; color: white; padding: 20px; border-radius: 10px; font-family: Arial; position: fixed; top: 50%; left: 50%; transform: translate(-50%, -50%); width: 80%; max-height: 80%; overflow: auto; z-index: 10000;">
        <h2 style="text-align: center; color: #4CAF50;">Résumé de l'extraction</h2>
        <p><b>Nombre total de voitures extraites:</b> {len(all_car_data)}</p>
        <p><b>Nombre de pages parcourues:</b> {page_count}</p>
        <h3>Top 5 des marques les plus fréquentes:</h3>
        <ul>
    """
    
    for make, count in sorted(makes_count.items(), key=lambda x: x[1], reverse=True)[:5]:
        summary_html += f"<li>{make}: {count} voitures</li>"
    
    summary_html += """
        </ul>
        <h3>Top 5 des carburants les plus fréquents:</h3>
        <ul>
    """
    
    for fuel, count in sorted(fuel_types.items(), key=lambda x: x[1], reverse=True)[:5]:
        summary_html += f"<li>{fuel}: {count} voitures</li>"
    
    summary_html += """
        </ul>
        <p style="text-align: center; margin-top: 20px;">
            Les données complètes ont été enregistrées dans le fichier 'autoscout24_complete_data.json'
        </p>
        <p style="text-align: center;">
            <button onclick="this.parentNode.parentNode.style.display='none';" 
                style="background-color: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer;">
                Fermer
            </button>
        </p>
    </div>
    """
    
    # Afficher le résumé
    driver.execute_script(f"document.body.insertAdjacentHTML('beforeend', `{summary_html}`);")

except Exception as e:
    print(f"Une erreur s'est produite: {e}")
    show_progress_message(f"ERREUR: {str(e)[:100]}...")
    
    # Enregistrer les données déjà collectées en cas d'erreur
    if all_car_data:
        with open("autoscout24_data_error_recovery.json", "w", encoding="utf-8") as json_file:
            json.dump(all_car_data, json_file, ensure_ascii=False, indent=2)
        print(f"Les données collectées jusqu'à l'erreur ont été sauvegardées dans 'autoscout24_data_error_recovery.json'")
        
        # Afficher un message d'erreur à l'écran
        error_html = f"""
        <div style="background-color: #ff4c4c; color: white; padding: 20px; border-radius: 10px; font-family: Arial; position: fixed; top: 50%; left: 50%; transform: translate(-50%, -50%); z-index: 10000;">
            <h2>Erreur lors de l'extraction</h2>
            <p>{str(e)}</p>
            <p><b>{len(all_car_data)}</b> voitures ont été extraites avant l'erreur.</p>
            <p>Les données ont été sauvegardées dans 'autoscout24_data_error_recovery.json'</p>
            <p style="text-align: center; margin-top: 20px;">
                <button onclick="this.parentNode.parentNode.style.display='none';" 
                    style="background-color: #333; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer;">
                    Fermer
                </button>
            </p>
        </div>
        """
        driver.execute_script(f"document.body.insertAdjacentHTML('beforeend', `{error_html}`);")

finally:
    # Supprimer le message de progression
    driver.execute_script("""
        var progressMsg = document.getElementById('scraping-progress-msg');
        if (progressMsg) {
            progressMsg.remove();
        }
    """)
    
    # Garder le navigateur ouvert pour inspection
    input("Appuyez sur Entrée pour fermer le navigateur...")
    driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import csv
import re

# Configuration du navigateur
options = Options()
options.add_argument("--start-maximized")
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

# Initialiser le navigateur
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)  # Attente explicite de 10 secondes maximum

# Fonction pour mettre en évidence un élément avec une bordure rouge
def highlight_element(element, duration=0.5):
    # Faire défiler jusqu'à l'élément pour qu'il soit visible
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
    time.sleep(0.2)  # Attendre que le défilement soit terminé
    
    # Ajouter une bordure rouge et un fond rouge transparent
    original_style = element.get_attribute('style')
    driver.execute_script("""
    arguments[0].setAttribute('style', arguments[1] + 
        '; border: 3px solid red; background: rgba(255, 0, 0, 0.2)');
    """, element, original_style)
    
    time.sleep(duration)  # Maintenir la mise en évidence
    
    # Restaurer le style original
    driver.execute_script("""
    arguments[0].setAttribute('style', arguments[1]);
    """, element, original_style)

# Fonction pour afficher un message de progression à l'écran
def show_progress_message(message):
    # Créer ou mettre à jour l'élément de message de progression
    driver.execute_script("""
        var progressMsg = document.getElementById('scraping-progress-msg');
        if (!progressMsg) {
            progressMsg = document.createElement('div');
            progressMsg.id = 'scraping-progress-msg';
            progressMsg.style.position = 'fixed';
            progressMsg.style.top = '10px';
            progressMsg.style.left = '10px';
            progressMsg.style.backgroundColor = 'rgba(0, 0, 0, 0.8)';
            progressMsg.style.color = 'white';
            progressMsg.style.padding = '10px';
            progressMsg.style.borderRadius = '5px';
            progressMsg.style.zIndex = '9999';
            progressMsg.style.fontSize = '16px';
            progressMsg.style.fontWeight = 'bold';
            document.body.appendChild(progressMsg);
        }
        progressMsg.textContent = arguments[0];
    """, message)

# Fonctions utiles pour l'extraction des données détaillées
def safe_text(by, identifier):
    try:
        element = driver.find_element(by, identifier)
        highlight_element(element, 0.3)
        return element.text.strip()
    except:
        return ""

def get_xpath_text(xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        highlight_element(element, 0.3)
        return element.text.strip()
    except:
        return ""

def extract_from_dt(dt_label):
    try:
        dt_elements = driver.find_elements(By.CLASS_NAME, "DataGrid_defaultDtStyle__soJ6R")
        for dt in dt_elements:
            if dt.text.strip().lower() == dt_label.lower():
                highlight_element(dt, 0.3)
                dd = dt.find_element(By.XPATH, "following-sibling::dd")
                highlight_element(dd, 0.3)
                return dd.text.strip()
    except:
        pass
    
    # Tentative alternative avec XPath
    try:
        xpath = f"//dt[contains(text(), '{dt_label}')]/following-sibling::dd[1]"
        element = driver.find_element(By.XPATH, xpath)
        highlight_element(element, 0.3)
        return element.text.strip()
    except:
        pass
    
    return ""

# Liste pour stocker toutes les données
all_car_data = []

# Paramètres pour le CSV
csv_filename = "autoscout24_details.csv"
fieldnames = [
    "URL", "Marque", "Modèle", "Prix", "Carrosserie", "Etat", "Sieges", "Portes", 
    "Annonce ID", "Garantie", "Kilometrage", "Annee", "Puissance", 
    "Transmission", "Cylindree", "Carburant", "CO2", "Couleur originale", 
    "Type peinture", "Couleur interieur", "position_page"
]

# Créer le fichier CSV et écrire l'en-tête
with open(csv_filename, mode="w", newline='', encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

try:
    # Accéder au site web
    driver.get("https://www.autoscout24.fr/")
    print("Page chargée avec succès")
    show_progress_message("Page chargée avec succès")
    time.sleep(3)
    
    # Gérer le popup de consentement spécifique à AutoScout24
    try:
        # Attendre que le popup de consentement apparaisse
        consent_popup = wait.until(EC.presence_of_element_located((By.ID, "as24-cmp-popup")))
        print("Popup de consentement détecté")
        show_progress_message("Popup de consentement détecté")
        
        # Accepter tous les cookies (bouton "Accepter tout")
        accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'consent-accept')]")))
        highlight_element(accept_button)
        accept_button.click()
        print("Cookies acceptés")
        show_progress_message("Cookies acceptés")
        time.sleep(2)
    except Exception as e:
        print(f"Erreur lors de la gestion du consentement: {e}")
        show_progress_message(f"Erreur: {e}")
    
    # Attendre que le bouton de recherche soit cliquable
    search_button = wait.until(EC.element_to_be_clickable((By.ID, "search-mask-search-cta")))
    print("Bouton de recherche trouvé")
    show_progress_message("Bouton de recherche trouvé")
    
    # Faire défiler jusqu'au bouton si nécessaire
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", search_button)
    time.sleep(1)
    
    # Mettre en évidence et cliquer sur le bouton
    highlight_element(search_button)
    search_button.click()
    print("Clic sur le bouton de recherche effectué")
    show_progress_message("Recherche de voitures en cours...")
    
    # Attendre que la page de résultats se charge
    wait.until(EC.url_contains("lst"))
    print("Page de résultats chargée")
    show_progress_message("Page de résultats chargée")
    
    # Boucle pour parcourir les pages
    page_count = 1
    max_pages = 20
    current_url = driver.current_url
    total_vehicles_processed = 0
    vehicle_urls = []
    
    while page_count <= max_pages:
        print(f"\n===== COLLECTE DES URLS DE LA PAGE {page_count} =====")
        show_progress_message(f"COLLECTE DES URLS - PAGE {page_count}/{max_pages}")
        
        # Attendre que le conteneur principal soit chargé
        print("Recherche du conteneur principal ListPage_main___0g2X...")
        main_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "main.ListPage_main___0g2X")))
        highlight_element(main_container, 1)
        print(f"Conteneur principal de la page {page_count} trouvé")
        
        # Récupérer tous les liens d'articles (voitures) dans le conteneur principal
        links = []
        try:
            # Première tentative: chercher les liens dans les articles
            articles = main_container.find_elements(By.CSS_SELECTOR, "article")
            for article in articles:
                try:
                    link = article.find_element(By.CSS_SELECTOR, "a.ListItem_title__ndA4s")
                    position_on_page = len(links) + 1
                    links.append((link.get_attribute("href"), position_on_page))
                    highlight_element(link, 0.2)
                except:
                    # Tentative alternative
                    try:
                        link = article.find_element(By.CSS_SELECTOR, "a")
                        if "offres" in link.get_attribute("href"):
                            position_on_page = len(links) + 1
                            links.append((link.get_attribute("href"), position_on_page))
                            highlight_element(link, 0.2)
                    except:
                        pass
        except:
            # Si aucun article n'est trouvé, chercher directement les liens
            links_elements = main_container.find_elements(By.CSS_SELECTOR, "a.ListItem_title__ndA4s")
            for i, link in enumerate(links_elements):
                position_on_page = i + 1
                links.append((link.get_attribute("href"), position_on_page))
                highlight_element(link, 0.2)
        
        print(f"Nombre de liens de véhicules trouvés sur la page {page_count}: {len(links)}")
        show_progress_message(f"Page {page_count}: {len(links)} liens trouvés")
        
        # Ajouter les liens à la liste globale
        vehicle_urls.extend(links)
        
        # Vérifier s'il y a une page suivante
        if page_count < max_pages:
            try:
                # Trouver le bouton "Suivant"
                next_button = None
                
                # Plusieurs tentatives pour trouver le bouton suivant
                try:
                    next_button = wait.until(EC.element_to_be_clickable((
                        By.XPATH, "//button[@aria-label='Aller à la page suivante']")))
                except:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((
                            By.XPATH, "//button[contains(., 'Suivant')]")))
                    except:
                        try:
                            next_button = wait.until(EC.element_to_be_clickable((
                                By.CSS_SELECTOR, "li.prev-next button")))
                        except:
                            next_button = wait.until(EC.element_to_be_clickable((
                                By.XPATH, "//svg[contains(@class, 'chevron-right')]/parent::button")))
                
                if next_button:
                    # Faire défiler jusqu'au bouton suivant
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_button)
                    time.sleep(1)
                    
                    # Mettre en évidence et cliquer sur le bouton suivant
                    highlight_element(next_button, 1)
                    print(f"Bouton 'Suivant' trouvé. Passage à la page {page_count + 1}...")
                    show_progress_message(f"Navigation vers la page {page_count + 1}...")
                    next_button.click()
                    
                    # Attendre que la nouvelle page se charge
                    time.sleep(3)
                    
                    # Vérifier que la page a bien changé
                    new_url = driver.current_url
                    if new_url != current_url:
                        current_url = new_url
                        print(f"URL actuelle après clic: {current_url}")
                        page_count += 1
                    else:
                        print("L'URL n'a pas changé après le clic, tentative supplémentaire...")
                        next_button.click()
                        time.sleep(3)
                        new_url = driver.current_url
                        if new_url != current_url:
                            current_url = new_url
                            page_count += 1
                        else:
                            print("Impossible de passer à la page suivante.")
                            break
                else:
                    print("Bouton 'Suivant' non trouvé.")
                    break
            except Exception as e:
                print(f"Impossible de passer à la page suivante: {e}")
                show_progress_message(f"Erreur de navigation: {e}")
                break
        else:
            print(f"Nombre maximum de pages ({max_pages}) atteint.")
            show_progress_message(f"Fin de la collecte des URLs - {max_pages} pages")
            break
    
    # Maintenant, visiter chaque URL et extraire les détails
    print(f"\n===== EXTRACTION DES DÉTAILS POUR {len(vehicle_urls)} VÉHICULES =====")
    
    for i, (url, position) in enumerate(vehicle_urls):
        progress_msg = f"Extraction véhicule {i+1}/{len(vehicle_urls)}"
        print(f"\nVisit de l'URL ({i+1}/{len(vehicle_urls)}): {url}")
        show_progress_message(progress_msg)
        
        try:
            # Visiter la page du véhicule
            driver.get(url)
            
            # Attendre que les détails du véhicule se chargent
            try:
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "StageTitle_container__vMhNz")))
            except:
                # Tentative alternative
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "VehicleOverview_container__")))
            
            time.sleep(1.5)  # Petit délai pour s'assurer que tout est chargé
            
            # Extraire les détails du véhicule
            # Tenter d'extraire la marque et le modèle
            marque_modele = get_xpath_text("//div[contains(@class,'StageTitle_makeModelContainer__')]//span").strip()
            modele_version = get_xpath_text("//div[contains(@class,'StageTitle_modelVersion__')]").strip()
            
            # Si les sélecteurs ci-dessus échouent, essayer d'autres sélecteurs
            # Si les sélecteurs ci-dessus échouent, essayer d'autres sélecteurs
            if not marque_modele:
                marque_modele = get_xpath_text("//h1[contains(@class, 'StageTitle')]")
                
                # Essayer de séparer la marque et le modèle
                marque = ""
                modele = marque_modele
                for marque_possible in ["Mercedes", "BMW", "Audi", "Volkswagen", "Renault", "Peugeot", "Citroen", "Toyota", "Ford", "Opel"]:
                    if marque_possible.lower() in marque_modele.lower():
                        marque = marque_possible
                        modele = marque_modele.replace(marque_possible, "").strip()
                        break
            else:
                marque = marque_modele
                modele = modele_version
                
            # Extraire les autres détails
            prix = safe_text(By.CLASS_NAME, "PriceInfo_price__XU0aF")
            if not prix:
                prix = get_xpath_text("//div[contains(@class, 'Price')]")
                if not prix:
                    prix = get_xpath_text("//p[contains(@data-item-name, 'price')]")
            
            # Extraire tous les détails techniques
            car_details = {
                "URL": url,
                "Marque": marque,
                "Modèle": modele,
                "Prix": prix,
                "Carrosserie": extract_from_dt("Carrosserie"),
                "Etat": extract_from_dt("État"),
                "Sieges": extract_from_dt("Sièges"),
                "Portes": extract_from_dt("Portes"),
                "Annonce ID": extract_from_dt("N° d'annonce"),
                "Garantie": extract_from_dt("Garantie"),
                "Kilometrage": safe_text(By.CLASS_NAME, "Carpass_carpassLink__qhOc1"),
                "Annee": extract_from_dt("Année") or extract_from_dt("Première immatriculation"),
                "Puissance": extract_from_dt("Puissance kW (CH)"),
                "Transmission": extract_from_dt("Transmission") or extract_from_dt("Boîte de vitesses"),
                "Cylindree": extract_from_dt("Cylindrée"),
                "Carburant": extract_from_dt("Carburant") or extract_from_dt("Type de carburant"),
                "CO2": extract_from_dt("Émissions de CO2") or extract_from_dt("Émission de CO₂"),
                "Couleur originale": extract_from_dt("Couleur originale") or extract_from_dt("Couleur"),
                "Type peinture": extract_from_dt("Type de peinture"),
                "Couleur interieur": extract_from_dt("La couleur de l'intérieur"),
                "position_page": position
            }
            
            # Si le kilométrage n'est pas trouvé via le sélecteur spécifique, essayer autrement
            if not car_details["Kilometrage"]:
                car_details["Kilometrage"] = extract_from_dt("Kilométrage")
            
            # Ajouter aux données globales
            all_car_data.append(car_details)
            print(f"Détails extraits pour le véhicule {i+1}")
            
            # Écrire dans le CSV pour chaque véhicule (pour éviter de perdre des données en cas d'erreur)
            with open(csv_filename, mode="a", newline='', encoding="utf-8") as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(car_details)
            
            # Ajouter également au fichier JSON
            with open(f"autoscout24_details_vehicle_{i+1}.json", "w", encoding="utf-8") as json_file:
                json.dump(car_details, json_file, ensure_ascii=False, indent=2)
            
            # Pause entre les requêtes pour éviter d'être bloqué
            time.sleep(1 + (i % 5) * 0.5)  # Pauses variables: 1s, 1.5s, 2s, 2.5s, 3s
            
        except Exception as e:
            print(f"Erreur lors de l'extraction des détails pour l'URL {url}: {e}")
            show_progress_message(f"Erreur sur véhicule {i+1}: {str(e)[:50]}...")
            continue
    
    # Enregistrer toutes les données dans un fichier JSON final
    with open("autoscout24_all_details.json", "w", encoding="utf-8") as json_file:
        json.dump(all_car_data, json_file, ensure_ascii=False, indent=2)
    
    print(f"\nToutes les données ont été extraites et enregistrées dans 'autoscout24_all_details.json' et '{csv_filename}'")
    print(f"Nombre total de véhicules extraits: {len(all_car_data)}")
    show_progress_message(f"TERMINÉ! {len(all_car_data)} véhicules extraits au total")
    
    # Afficher un résumé des données
    makes_count = {}
    models_count = {}
    fuel_types = {}
    years = {}
    
    for car in all_car_data:
        make = car.get("Marque", "Inconnu")
        model = car.get("Modèle", "Inconnu")
        fuel = car.get("Carburant", "Inconnu")
        year = car.get("Annee", "Inconnu")
        
        makes_count[make] = makes_count.get(make, 0) + 1
        models_count[model] = models_count.get(model, 0) + 1
        fuel_types[fuel] = fuel_types.get(fuel, 0) + 1
        years[year] = years.get(year, 0) + 1
    
    # Afficher un résumé à l'écran
    summary_html = f"""
    <div style="background-color: #333; color: white; padding: 20px; border-radius: 10px; font-family: Arial; position: fixed; top: 50%; left: 50%; transform: translate(-50%, -50%); width: 80%; max-height: 80%; overflow: auto; z-index: 10000;">
        <h2 style="text-align: center; color: #4CAF50;">Résumé de l'extraction</h2>
        <p><b>Nombre total de véhicules extraits:</b> {len(all_car_data)}</p>
        <p><b>Nombre de pages parcourues:</b> {page_count}</p>
        <h3>Top 5 des marques les plus fréquentes:</h3>
        <ul>
    """
    
    for make, count in sorted(makes_count.items(), key=lambda x: x[1], reverse=True)[:5]:
        if make:
            summary_html += f"<li>{make}: {count} véhicules</li>"
    
    summary_html += """
        </ul>
        <h3>Top 5 des carburants les plus fréquents:</h3>
        <ul>
    """
    
    for fuel, count in sorted(fuel_types.items(), key=lambda x: x[1], reverse=True)[:5]:
        if fuel:
            summary_html += f"<li>{fuel}: {count} véhicules</li>"
    
    summary_html += """
        </ul>
        <h3>Répartition par année:</h3>
        <ul>
    """
    
    for year, count in sorted(years.items(), key=lambda x: (x[0] if x[0] else "0"), reverse=True)[:10]:
        if year:
            summary_html += f"<li>{year}: {count} véhicules</li>"
    
    summary_html += """
        </ul>
        <p style="text-align: center; margin-top: 20px;">
            Les données complètes ont été enregistrées dans les fichiers 'autoscout24_all_details.json' et 'autoscout24_details.csv'
        </p>
        <p style="text-align: center;">
            <button onclick="this.parentNode.parentNode.style.display='none';" 
                style="background-color: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer;">
                Fermer
            </button>
        </p>
    </div>
    """
    
    # Afficher le résumé
    driver.execute_script(f"document.body.insertAdjacentHTML('beforeend', `{summary_html}`);")

except Exception as e:
    print(f"Une erreur s'est produite: {e}")
    show_progress_message(f"ERREUR: {str(e)[:100]}...")
    
    # Enregistrer les données déjà collectées en cas d'erreur
    if all_car_data:
        with open("autoscout24_error_recovery.json", "w", encoding="utf-8") as json_file:
            json.dump(all_car_data, json_file, ensure_ascii=False, indent=2)
        print(f"Les données collectées jusqu'à l'erreur ont été sauvegardées dans 'autoscout24_error_recovery.json'")
        
        # Afficher un message d'erreur à l'écran
        error_html = f"""
        <div style="background-color: #ff4c4c; color: white; padding: 20px; border-radius: 10px; font-family: Arial; position: fixed; top: 50%; left: 50%; transform: translate(-50%, -50%); z-index: 10000;">
            <h2>Erreur lors de l'extraction</h2>
            <p>{str(e)}</p>
            <p><b>{len(all_car_data)}</b> véhicules ont été extraits avant l'erreur.</p>
            <p>Les données ont été sauvegardées dans 'autoscout24_error_recovery.json'</p>
            <p style="text-align: center; margin-top: 20px;">
                <button onclick="this.parentNode.parentNode.style.display='none';" 
                    style="background-color: #333; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer;">
                    Fermer
                </button>
            </p>
        </div>
        """
        driver.execute_script(f"document.body.insertAdjacentHTML('beforeend', `{error_html}`);")

finally:
    # Supprimer le message de progression
    driver.execute_script("""
        var progressMsg = document.getElementById('scraping-progress-msg');
        if (progressMsg) {
            progressMsg.remove();
        }
    """)
    
    # Garder le navigateur ouvert pour inspection
    input("Appuyez sur Entrée pour fermer le navigateur...")
    driver.quit()